In [51]:
#Execução sem otimização

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Carregar o dataset
data = pd.read_csv('powerconsumption.csv')

# Separar as features e o target
X = data.drop(['Datetime', 'PowerConsumption_Zone1'], axis=1)  # Exclua outras zonas se necessário
y = data['PowerConsumption_Zone1']

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar os dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Construir a rede neural
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='linear')  # Saída linear para regressão
])

# Compilar o modelo
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Treinar o modelo
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test).flatten()

# Avaliar o modelo
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"R² Score: {r2:.4f}")


Epoch 1/100


c:\Users\emanu\my-project\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1049/1049 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1044837952.0000 - mae: 31520.2773 - val_loss: 476769184.0000 - val_mae: 20826.6348
Epoch 2/100
1049/1049 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 327786848.0000 - mae: 16294.2568 - val_loss: 97243240.0000 - val_mae: 8275.9746
Epoch 3/100
1049/1049 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 120832576.0000 - mae: 9057.6348 - val_loss: 47798164.0000 - val_mae: 5694.9116
Epoch 4/100
1049/1049 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 82068616.0000 - mae: 7261.8848 - val_loss: 25798288.0000 - val_mae: 4067.2222
Epoch 5/100
1049/1049 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 63193664.0000 - mae: 6245.8174 - val_loss: 16917134.0000 - val_mae: 3237.7651
Epoch 6/100
1049/1049 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 54922448.0000 - mae: 5842.3091 - val_loss: 13321447.0000 - val_mae: 2860.8333
Epoch 7/100
1049/1049 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 51770616.0000 - mae: 5645.3604 - val_loss: 11557613.0000 - val_mae: 2648.5586
Epoch 8/100
1049/

In [56]:
#Execuução de 114 minutos com warning

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1_l2
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Carregar o dataset
data = pd.read_csv('powerconsumption.csv')

# Separar as features e o target
X = data.drop(['Datetime', 'PowerConsumption_Zone1'], axis=1)  # Exclua outras zonas se necessário
y = data['PowerConsumption_Zone1']

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar os dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Função para criar o modelo com regularização L1/L2
def create_model(optimizer='adam', dropout_rate=0.3, units=64, l1_penalty=0.01, l2_penalty=0.01):
    model = Sequential([
        Dense(units, input_dim=X_train.shape[1], activation='relu', kernel_regularizer=l1_l2(l1_penalty, l2_penalty)),
        Dropout(dropout_rate),
        Dense(units // 2, activation='relu', kernel_regularizer=l1_l2(l1_penalty, l2_penalty)),
        Dropout(dropout_rate),
        Dense(1, activation='linear')  # Saída linear para regressão
    ])
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

# Parâmetros para o Grid Search
param_grid = {
    'optimizer': ['adam', 'rmsprop'],
    'dropout_rate': [0.2, 0.3, 0.4],
    'units': [64, 128, 256],
    'l1_penalty': [0.01, 0.1, 0.001],
    'l2_penalty': [0.01, 0.1, 0.001]
}

# Função para treinar o modelo (para uso com GridSearch)
def model_train(params):
    model = create_model(optimizer=params['optimizer'], 
                         dropout_rate=params['dropout_rate'], 
                         units=params['units'], 
                         l1_penalty=params['l1_penalty'], 
                         l2_penalty=params['l2_penalty'])
    
    # Treinar o modelo
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
    
    # Fazer previsões no conjunto de teste
    y_pred = model.predict(X_test).flatten()
    
    # Calcular as métricas
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    return {'loss': mse, 'status': 'ok', 'mae': mae, 'r2': r2}

# Usando GridSearch manualmente com o código de treinamento
from sklearn.model_selection import ParameterGrid

# Criando a grade de parâmetros para a busca
param_grid = {
    'optimizer': ['adam', 'rmsprop'],
    'dropout_rate': [0.2, 0.3, 0.4],
    'units': [64, 128],
    'l1_penalty': [0.01, 0.1],
    'l2_penalty': [0.01, 0.1]
}

# Testando todas as combinações de parâmetros
best_mae = float('inf')
best_params = None

for params in ParameterGrid(param_grid):
    result = model_train(params)
    if result['mae'] < best_mae:
        best_mae = result['mae']
        best_params = params

print(f"Melhores parâmetros: {best_params}")
print(f"Melhor MAE: {best_mae}")


c:\Users\emanu\my-project\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 786us/step


c:\Users\emanu\my-project\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 747us/step


c:\Users\emanu\my-project\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 841us/step


c:\Users\emanu\my-project\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 684us/step


c:\Users\emanu\my-project\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 708us/step
Melhores parâmetros: {'dropout_rate': 0.2, 'l1_penalty': 0.1, 'l2_penalty': 0.1, 'optimizer': 'rmsprop', 'units': 128}
Melhor MAE: 1547.1225019945869


In [60]:

best_mae = result['mae']
best_mse = result['loss']  # O 'loss' é o MSE
best_r2 = result['r2']
best_params = params

# Imprimir os melhores resultados
print(f"Últimos parâmetros: {best_params}")
print(f"Últimos MAE: {best_mae}")
print(f"Últimos MSE: {best_mse}")
print(f"Últimos R²: {best_r2}")


Melhores parâmetros: {'dropout_rate': 0.2, 'l1_penalty': 0.01, 'l2_penalty': 0.1, 'optimizer': 'adam', 'units': 128}
Melhor MAE: 1977.7425816812524
Melhor MSE: 6747264.431293927
Melhor R²: 0.8663759496533805


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1_l2
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Carregar o dataset
data = pd.read_csv('powerconsumption.csv')

# Separar as features e o target
X = data.drop(['Datetime', 'PowerConsumption_Zone1'], axis=1)  # Exclua outras zonas se necessário
y = data['PowerConsumption_Zone1']

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar os dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Função para criar o modelo com regularização L1/L2
def create_model(optimizer='adam', dropout_rate=0.3, units=64, l1_penalty=0.01, l2_penalty=0.01):
    model = Sequential([
        Input(shape=(X_train.shape[1],)),  # Definindo a entrada como um Input layer
        Dense(units, activation='relu', kernel_regularizer=l1_l2(l1_penalty, l2_penalty)),
        Dropout(dropout_rate),
        Dense(units // 2, activation='relu', kernel_regularizer=l1_l2(l1_penalty, l2_penalty)),
        Dropout(dropout_rate),
        Dense(1, activation='linear')  # Saída linear para regressão
    ])
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

# Parâmetros para o Grid Search
param_grid = {
    'optimizer': ['adam', 'rmsprop'],
    'dropout_rate': [0.2, 0.3, 0.4],
    'units': [64, 128, 256],
    'l1_penalty': [0.01, 0.1, 0.001],
    'l2_penalty': [0.01, 0.1, 0.001]
}

# Função para treinar o modelo (para uso com GridSearch)
def model_train(params):
    model = create_model(optimizer=params['optimizer'],
                         dropout_rate=params['dropout_rate'],
                         units=params['units'],
                         l1_penalty=params['l1_penalty'],
                         l2_penalty=params['l2_penalty'])

    # Treinar o modelo
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)

    # Fazer previsões no conjunto de teste
    y_pred = model.predict(X_test).flatten()

    # Calcular as métricas
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    return {'loss': mse, 'status': 'ok', 'mae': mae, 'mse': mse, 'r2': r2}

# Usando GridSearch manualmente com o código de treinamento
from sklearn.model_selection import ParameterGrid

# Criando a grade de parâmetros para a busca
param_grid = {
    'optimizer': ['adam', 'rmsprop'],
    'dropout_rate': [0.2, 0.3, 0.4],
    'units': [64, 128],
    'l1_penalty': [0.01, 0.1],
    'l2_penalty': [0.01, 0.1]
}

# Testando todas as combinações de parâmetros
best_mae = float('inf')
best_mse = float('inf')
best_r2 = -float('inf')
best_params = None

for params in ParameterGrid(param_grid):
    result = model_train(params)
    if result['mae'] < best_mae:
        best_mae = result['mae']
        best_mse = result['mse']
        best_r2 = result['r2']
        best_params = params

# Exibindo os melhores parâmetros e as métricas
print(f"Melhores parâmetros: {best_params}")
print(f"Melhor MAE: {best_mae}")
print(f"Melhor MSE: {best_mse}")
print(f"Melhor R²: {best_r2}")


328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 688us/step
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 706us/step
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 666us/step
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 658us/step
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 640us/step
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 655us/step
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 664us/step
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 669us/step
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 651us/step
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 649us/step
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 651us/step
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 741us/step
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 637us/step
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 705us/step
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 673us/step
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 658us/step
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 665us/step
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 683us/step
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 656us/step
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 679us/step
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 681us/step
328/328 ━━━━━

In [ ]:
#Com timer de 10 minutos

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.callbacks import EarlyStopping, Callback
import time
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import kagglehub

# Baixar a versão mais recente do dataset
path = kagglehub.dataset_download("fedesoriano/electric-power-consumption")

# Suponha que o dataset seja um arquivo CSV
dataset_path = f"{path}/powerconsumption.csv"

# Carregar os dados
data = pd.read_csv(dataset_path)

# Separar as features e o target
X = data.drop(['Datetime', 'PowerConsumption_Zone1'], axis=1)
y = data['PowerConsumption_Zone1']

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar os dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Função para criar o modelo com regularização L1/L2
def create_model(optimizer='adam', dropout_rate=0.3, units=64, l1_penalty=0.01, l2_penalty=0.01):
    model = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(units, activation='relu', kernel_regularizer=l1_l2(l1_penalty, l2_penalty)),
        Dropout(dropout_rate),
        Dense(units // 2, activation='relu', kernel_regularizer=l1_l2(l1_penalty, l2_penalty)),
        Dropout(dropout_rate),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

# Callback para controlar o tempo máximo de treinamento
class TimeStopping(Callback):
    def __init__(self, max_duration_minutes=10):
        super(TimeStopping, self).__init__()
        self.max_duration = max_duration_minutes * 60
        self.start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        """Verifica se o tempo total máximo foi alcançado no final de cada época."""
        elapsed_time = time.time() - self.start_time
        if elapsed_time >= self.max_duration:
            print(f"\nTempo limite de {self.max_duration / 60} minutos alcançado. Parando o treinamento...")
            self.model.stop_training = True

# Função para treinar o modelo e calcular métricas
def model_train(params, time_stopping):
    model = create_model(optimizer=params['optimizer'],
                         dropout_rate=params['dropout_rate'],
                         units=params['units'],
                         l1_penalty=params['l1_penalty'],
                         l2_penalty=params['l2_penalty'])

    # Adicionar callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Treinar o modelo com os callbacks
    model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2,
              verbose=0, callbacks=[early_stopping, time_stopping])

    # Previsões e métricas
    y_pred = model.predict(X_test).flatten()
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    return {'loss': mse, 'mae': mae, 'mse': mse, 'r2': r2}

# Configuração do Grid Search
param_grid = {
    'optimizer': ['adam', 'rmsprop'],
    'dropout_rate': [0.2, 0.3, 0.4],
    'units': [64, 128],
    'l1_penalty': [0.01, 0.1],
    'l2_penalty': [0.01, 0.1]
}

# Iniciar o controle de tempo
start_time = time.time()
max_duration = 10 * 60  # 10 minutos

best_mae = float('inf')
best_params = None
best_mse = None
best_r2 = None

# Rodar GridSearch
for params in ParameterGrid(param_grid):
    print(f"Tentando parâmetros: {params}")
    
    # Callback para controlar o tempo durante o treinamento
    time_stopping = TimeStopping(max_duration_minutes=(max_duration - (time.time() - start_time)) / 60)

    result = model_train(params, time_stopping)
    
    print(f"Resultado: MAE={result['mae']:.4f}, MSE={result['mse']:.4f}, R²={result['r2']:.4f}")
    
    # Checar se o tempo total de treinamento excedeu
    elapsed_time = time.time() - start_time
    if elapsed_time >= max_duration:
        print(f"\nTempo máximo de {max_duration / 60} minutos alcançado. Finalizando otimização...")
        break

    if result['mae'] < best_mae:
        best_mae = result['mae']
        best_params = params
        best_mse = result['mse']
        best_r2 = result['r2']

# Exibir os melhores resultados
print("\nOtimização concluída!")
print(f"Melhores parâmetros: {best_params}")
print(f"Melhor MAE: {best_mae:.4f}")
print(f"Melhor MSE: {best_mse:.4f}")
print(f"Melhor R²: {best_r2:.4f}")